In [1]:
import os 
from datasets import load_dataset
import nltk.data
import json

data_files = []
dirname = "/Users/marisa/clausal-coordinate-ellipsis/german-common-crawl/correct_files"
for filename in os.listdir(dirname):
    data_files.append(dirname + "/" + filename)

ds = load_dataset("json", data_files=data_files, split='train', streaming=True)
ds = ds.select_columns('raw_content')

In [2]:
def tokenize(text):
    sents = nltk.sent_tokenize(text['raw_content'])
    for s in sents:
        s = s + '[NEXT]'
    text['raw_content'] = str(sents)
    return text

In [3]:
ds_modified = ds.map(tokenize)
ds_modified

IterableDataset({
    features: Unknown,
    n_shards: 5
})

## Datensatz nach json schreiben

In [6]:
# all together
sent_id = 0
with open("de_de_pairs.jsonl", "a") as file:
    for tokenized_sentence in ds_modified:
        splitted = tokenized_sentence['raw_content'].split("',")
        for i in range(len(splitted)): # wenn 1. x: 3 vorne entfernen, sonst 2, am Ende immer zwei
            splitted[i] = splitted[i][2:(len(splitted[i])-1)]
            jsn = {'id:': sent_id, 'sentence': splitted[i], 'gold_sentence': splitted[i]}
            json.dumps(jsn)
            file.write(str(jsn) + '\n')
            sent_id += 1
            if sent_id % 10000 == 0:
                print(sent_id)
        if sent_id > 1000000:
            break 

KeyboardInterrupt: 

In [ ]:
# CHECK: json Datei schreiben mit: ID, Satz, Satz_gold (Sätze aus gesamten Datensatz) 
# TODO: neuen Datensatz aus json laden und damit Model trainieren